<a href="https://colab.research.google.com/github/betoborda/pucp-pry-analisis-datos/blob/main/dashboard_ipywidgets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import functools
import io

import IPython
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor

# ipywidgets
import ipywidgets as widgets

from ipywidgets import Button, Label, Image
from ipywidgets import Box, VBox, HBox
from ipywidgets import AppLayout, Layout

In [ ]:
# Variables globales sobre el dataset

np.random.seed(42)

model = {}

num_columns = ['COSTO ALQUILER DIARIO', 'TOTAL ALQUILER', 'COSTO CONSERJE DIARIO', 
               'TOTAL CONSERJE' ,'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES',
               'LIQUIDACION EN DOLARES']

model["df_original"] = None
model["df_final"] = None
model["df"] = None
model["main_output"] = None
model["num_columns"] = num_columns

In [ ]:
###########
#. Errores (TODO)
###########

download_btn = widgets.Button(
    description='Descargar',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

#Widgets para dataframe errores
# TODO renombar variable output
df_errores = widgets.Output()

#funcion graficar pie
def show_error_graph(pie_output):
  global model

  with pie_output:
    clear_output()
    dat = [model['df'].shape[0], model['df_final'].shape[0]]
    index=['Errores','Validos']
    plt.pie(dat, labels=index,autopct='%1.1f%%',)
    plt.title('CALIDAD DE LA DATA')
    plt.show()


#Funcionalidad N°8: Mostrar en el otro tab df los errores (esta dentro de la funcion reglas)
def show_errors(df):
    columns = ['CIUDAD', 'TIPO', 'CATEGORIA', 'MONEDA', 'DIAS ALQUILER', 'TOTAL ALQUILER', 
               'TOTAL CONSERJE', 'COSTOS ADICIONALES SOLES', 'LIQUIDACION EN SOLES', 
               'LIQUIDACION EN DOLARES', 'ESTADO TRANSACCION']
    with df_errores:
        clear_output()
        display(df[columns])


def download_file(d):
    print("download df.xlsx", df.shape)
    df.to_excel("df.xlsx")


# Subscribir eventos
download_btn.on_click(download_file)


calidad_items = [df_errores]
calidad_column = widgets.VBox(calidad_items)

error_tab = widgets.VBox(
    children=[
        widgets.Label("Errores destacados"),
        download_btn,
        calidad_column]
)

In [ ]:
##############
# Carga Page
##############

def process_rules(checkboxes):
  global model

  df = model['df_original'].copy()

  if checkboxes[0].value:
    df['valcostoalquiler_REGLA1'] = np.where(~np.isclose(df['COSTO ALQUILER DIARIO']*df['DIAS ALQUILER'],df['TOTAL ALQUILER']), 1, 0)
  else:
    df['valcostoalquiler_REGLA1'] = 0
    
  if checkboxes[1].value:
    df['valcostoconserje_REGLA2'] = np.where(~np.isclose(df['DIAS CONSERJE']*df['COSTO CONSERJE DIARIO'],df['TOTAL CONSERJE']), 1, 0)
  else:
    df['valcostoconserje_REGLA2'] = 0

  if checkboxes[2].value:
    df['valdiasalquiler_REGLA3'] = np.where(~np.isclose(df['DIAS ALQUILER'], (df['CHECK OUT'] - df['CHECK IN']).dt.ceil('d').dt.days), 1, 0)
  else:
    df['valdiasalquiler_REGLA3'] = 0

  if checkboxes[3].value:
    df['valcheckout_REGLA4'] = np.where((df['CHECK OUT'].isnull()) & (df['ESTADO TRANSACCION']!='ANULADO'), 1, 0)
  else:
    df['valcheckout_REGLA4'] = 0

  if checkboxes[4].value:
    df['valadicionalusd_REGLA5'] = np.where((df['MONEDA']=='USD') & (df['COSTOS ADICIONALES SOLES']!=0), 1, 0)
  else:
    df['valadicionalusd_REGLA5'] = 0

  if checkboxes[5].value:
    df['valsumacostosusd_REGLA6'] = np.where((df['MONEDA']=='USD') & ~np.isclose(df['TOTAL ALQUILER']+df['TOTAL CONSERJE'],df['LIQUIDACION EN DOLARES']), 1, 0)
  else:
    df['valsumacostosusd_REGLA6'] = 0

  if checkboxes[6].value:
    df['valsumacostospen_REGLA7'] = np.where((df['MONEDA']=='PEN') & (df['TIPO']=='ALQUILER') & ~np.isclose(df['TOTAL ALQUILER']+df['TOTAL CONSERJE']+df['COSTOS ADICIONALES SOLES'],df['LIQUIDACION EN SOLES']), 1, 0)
  else:
    df['valsumacostospen_REGLA7'] = 0

  if checkboxes[7].value:
    df['valpenusdpen_REGLA8'] = np.where((df['MONEDA']=='PEN / USD') & (df['TIPO']=='ALQUILER') & ~np.isclose(df['COSTOS ADICIONALES SOLES'],df['LIQUIDACION EN SOLES']), 1, 0)
  else:
    df['valpenusdpen_REGLA8'] = 0

  if checkboxes[8].value:
    df['valpenusdusd_REGLA9'] = np.where((df['MONEDA']=='PEN / USD') & (df['TIPO']=='ALQUILER') & ~np.isclose(df['TOTAL ALQUILER'] + df['TOTAL CONSERJE'],df['LIQUIDACION EN DOLARES']), 1, 0)
  else:
    df['valpenusdusd_REGLA9'] = 0
  
  df['errores'] = np.where((df['valcostoalquiler_REGLA1']==1) | 
                           (df['valcostoconserje_REGLA2']==1) | 
                           (df['valdiasalquiler_REGLA3']==1) | 
                           (df['valcheckout_REGLA4']==1) | 
                           (df['valadicionalusd_REGLA5']==1) | 
                           (df['valsumacostosusd_REGLA6']==1) | 
                           (df['valsumacostospen_REGLA7']==1) | 
                           (df['valpenusdpen_REGLA8']==1)| 
                           (df['valpenusdusd_REGLA9']==1), 1, 0)

  # TODO actualizar logica
  model['df_final'] = df[df['errores']==0].iloc[:,0:19] ### Df limpia

  df = df[df['errores']==1].iloc[:,0:19] #### Df con errores
  model['df'] = df


def on_choose_file_change(change):
  pass


def on_upload_file_clicked(b, choose_file_btn, filename_txt):
  global model

  def leerArchivoCSV(botonCargar, nom_arch):
    return pd.read_excel(io.BytesIO(botonCargar.value[nom_arch]['content']))

  csv_nombre = next(iter(choose_file_btn.value))
  model["df_original"]=leerArchivoCSV(choose_file_btn, csv_nombre)
  filename_txt.value=csv_nombre


def on_process_file_clicked(b, rule_chk_list, pie_out):
  process_rules(rule_chk_list)
  #process_imputers()
  show_error_graph(pie_out)
  # TODO mostrar los errores del procesamiento en la otra pestaña
  #show_errors(df) 


def create_carga_buttons():
  choose_file_btn = widgets.FileUpload(
    accept='xlsx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    icon = 'upload'
  )
  file_name_txt = widgets.Text(
    placeholder='Ningún archivo cargado',
    disabled=True
  )
  upload_file_btn = widgets.Button(
    description='subir archivo',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check' # (FontAwesome names without the `fa-` prefix)
  )
  process_file_btn = widgets.Button(
    description='Procesar',
    button_style='success')

  upload_file_btn.on_click(functools.partial(on_upload_file_clicked, 
                                             choose_file_btn=choose_file_btn,
                                             filename_txt=file_name_txt))
  
  return widgets.HBox([choose_file_btn, 
                      file_name_txt, 
                      upload_file_btn, 
                      process_file_btn])


# panel = pestaña
def create_carga_tab_panel():

  rule_names = ['R1: total alquiler = días alquiler * costo alquiler diario', 
                'R2: total conserje = días conserje * costo conserje diario', 
                'R3: días alquiler = check out - check in', 
                'R4: Si check out vacío => estado transacción = Anulado', 
                'R5: Si moneda USD => no hay costos adicionales soles', 
                'R6: Si moneda USD => Total alquiler + Total conserje == liquidación en dólares', 
                'R7: Si moneda PEN => Total alquiler + Total conserje + costos adicionales soles == liquidación soles', 
                'R8: Si moneda PEN/USD => costos adicionales soles == liquidación en soles', 
                'R9: Si moneda PEN/USD => Total alquiler + Total conserje == liquidación en dólares']

  def create_checkbox(description):
    #TODO crear un un checkbox, un label para la regla y agruparlos
    return widgets.Checkbox(value=True, description=description, indent=False)

  rule_checkboxes = [create_checkbox(description=rule_name) for rule_name in rule_names]

  consistencia_column = widgets.VBox(children=[widgets.Label("Consistencia"),
                                     *rule_checkboxes], 
                                     layout=Layout(width='auto'))

  limpieza_items = [widgets.Label("Limpieza"), 
                    widgets.Checkbox(description="Nulos", disabled=True) ]
  limpieza_column = widgets.VBox(limpieza_items)

  #Widgets para pie
  pie_output = widgets.Output()

  calidad_items = [widgets.Label("Calidad de los datos"), pie_output]
  calidad_column = widgets.VBox(calidad_items)

  carga_buttons_hbox = create_carga_buttons()
  process_file_btn = carga_buttons_hbox.children[3]
  process_file_btn.on_click(functools.partial(on_process_file_clicked,
                                              rule_chk_list=rule_checkboxes,
                                              pie_out=pie_output))

  return widgets.VBox(
    children=[
        carga_buttons_hbox,
        widgets.HBox(
            children=[consistencia_column, 
                      limpieza_column, 
                      calidad_column],
            layout=Layout(width='auto')
        )
    ]
  )


def create_carga_page():
  # TODO añadir la pestaña error
  #children = [data_tab, error_tab]
  children = [create_carga_tab_panel()]
  
  carga_tab = widgets.Tab()
  carga_tab.children = children
  carga_tab.set_title(0, 'Data')
  carga_tab.set_title(1, 'Errores')
  
  return VBox([Label(value="Carga de Datos"), 
               carga_tab])


In [ ]:
################
# Outlier Page
################

def on_outlier_dropdown_x_changed(change, dropdown_y, outlier_output):
    global model

    # Hack: porque el método se invoca muchas veces
    if change['name'] != 'value':
      return

    x_idx = model["num_columns"].index(change.new)
    y_idx = model["num_columns"].index(dropdown_y.value)
    show_outlier_graph(outlier_output, x_idx, y_idx)


def on_outlier_dropdown_y_changed(change, dropdown_x, outlier_output):
    global model

    # Hack: porque el método se invoca muchas veces
    if change['name'] != 'value':
      return

    x_idx = model["num_columns"].index(dropdown_x.value)
    y_idx = model["num_columns"].index(change.new)
    show_outlier_graph(outlier_output, x_idx, y_idx)


def show_outlier_graph(outlier_output, x_idx, y_idx):
  global model
  
  df = model['df']
  ct = ColumnTransformer([('entre-0-1', StandardScaler(), num_columns)], 
                         remainder='drop')
  X = ct.fit_transform(df)

  clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

  y_pred = clf.fit_predict(X)
  X_scores = clf.negative_outlier_factor_

  with outlier_output:
    clear_output()
    plt.title("Local Outlier Factor (LOF)")
    plt.scatter(X[:, x_idx], X[:, y_idx], color='k', s=3., label='Data points')

    radius = (X_scores.max() - X_scores) / (X_scores.max() - X_scores.min())
    plt.scatter(X[:, x_idx], X[:, y_idx], s=1000 * radius, edgecolors='r', 
              facecolors='none', label='Outlier scores')
    plt.axis('tight')
    plt.xlim((-5, 5))
    plt.ylim((-5, 5))
    # plt.xlabel("prediction errors: %d" % (n_errors))
    plt.xlabel("prediction errors")
    legend = plt.legend(loc='upper left')
    legend.legendHandles[0]._sizes = [10]
    legend.legendHandles[1]._sizes = [20]
    plt.show()


def create_outlier_page():

  outlier_graph_x_dropdown = widgets.Dropdown(
    options=num_columns,
    value=num_columns[0],
    description='X field:',
  )

  outlier_graph_y_dropdown = widgets.Dropdown(
    options=num_columns,
    value=num_columns[1],
    description='Y field:',
  )
  outlier_scatter_output = widgets.Output()


  outlier_graph_x_dropdown.observe(functools.partial(on_outlier_dropdown_x_changed,
                                              dropdown_y=outlier_graph_y_dropdown,
                                              outlier_output=outlier_scatter_output
                                              ))

  outlier_graph_y_dropdown.observe(functools.partial(on_outlier_dropdown_y_changed,
                                              dropdown_x=outlier_graph_x_dropdown,
                                              outlier_output=outlier_scatter_output
                                              ))
  # default preview
  show_outlier_graph(outlier_scatter_output, 
                     model["num_columns"].index(num_columns[0]), 
                     model["num_columns"].index(num_columns[1]))

  outlier_page = widgets.VBox(
    children=[
        widgets.HBox(
            children=[Label(value="Parameters"),
                      outlier_graph_x_dropdown, 
                      outlier_graph_y_dropdown]
          ),
        outlier_scatter_output
      ]
    )

  return VBox([Label(value="Análisis de Outliers"), 
               outlier_page])


In [ ]:
##############
# Main layout
##############

def create_logo_pucp_image():
  logo_url = "https://dci.pucp.edu.pe/wp-content/uploads/2014/02/Logotipo_colores-290x145.jpg"
  logo_image = IPython.display.Image(logo_url, 
                                     width=290)
  return widgets.Image(value=logo_image.data, 
                       format='jpg',
                       width=290,
                       height=145)


def create_main_title():
  title = "Detección de patrones de alquiler por ciudad"
  title_layout = Layout(width='60%', 
                        display='flex', 
                        flex_flow='column', 
                        align_items='center', 
                        justify_content='center')
  return widgets.Label(value=title, 
                       layout=title_layout)


def create_global_header():
  return HBox([create_main_title(), 
               create_logo_pucp_image()])


# Main output
def create_content_output():
  main_output = widgets.Output(layout={'border': '1px solid black'})
  model["main_output"] = main_output
  return main_output


#  Main Menu
def show_load_page(b):
  with model["main_output"]:
    clear_output()
    display(create_carga_page())

def show_correlation_page(b):
  with model["main_output"]:
    clear_output()
    display(corr_section)

def show_outliers_page(b):
  with model["main_output"]:
    clear_output()
    display(create_outlier_page())


def create_global_menu():
  items_layout = Layout(width='auto') # override the default width of the button to 'auto' to let the button grow
  main_menu_layout = Layout(display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            border='solid',
                            width='90%')
  
  menu_options = ['Carga', 'Correlación', 'Outliers']
  main_menu_buttons = [Button(description=menu_option, layout=items_layout, button_style='success') for menu_option in menu_options]

  main_menu_buttons[0].on_click(show_load_page)
  main_menu_buttons[1].on_click(show_correlation_page)
  main_menu_buttons[2].on_click(show_outliers_page)

  return Box(children=main_menu_buttons, 
             layout=main_menu_layout)

In [ ]:
corr_section = VBox([Label(value="Correlación de datos")], 
                    layout=Layout())

AppLayout(header=create_global_header(),
          left_sidebar=create_global_menu(),
          center=create_content_output(),
          pane_widths=[1, 5, 0])

AppLayout(children=(HBox(children=(Label(value='Detección de patrones de alquiler por ciudad', layout=Layout(a…